# Imports

In [ ]:
from modules.data import data_loader, data_splitter, data_reader
from modules.models import arima, lag_llama
from modules.evaluation.evaluate import evaluate
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from itertools import islice

# Parameters

In [ ]:
PREDICTION_LENGTH = 1
FREQUENCY = "daily"
TYPE_OF_DATA = "stock"

# Loading the data

In [ ]:
data = data_reader.read_data(TYPE_OF_DATA)[0]

# Prepare the data

# Models

In [1]:
# arima


In [2]:
# lag llama

# Evaluation